# Text Mining Group Project


### Imports

In [2]:
#Don't forget to 'pip install tranformers, pip install torch' in conda prompt
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Data Acquisition

In [3]:
import json
#https://eightportions.com/datasets/Recipes/#fn:1 where the data is from
#scraped from 
#Foodnetwork.com, Epicurious.com, Allrecipes.com by someone

allr = open('recipes_raw_nosource_ar.json')
epi = open('recipes_raw_nosource_epi.json')
food = open('recipes_raw_nosource_fn.json')

data1 = json.load(allr) #data 1 has a lot of the word ADVERTISEMENT
data2 = json.load(epi) #data 2 looks good
data3 = json.load(food) #this too

#this load module loads the data into a dictionary 

for key, value in data3.items():
    print(value)
#example of a specific recipe with a certain key and the value is also a dictionary
print(data3["p3pKOD6jIHEcjf20CCXohP8uqkG5dGi"])

#example of only instructions
print(data3["p3pKOD6jIHEcjf20CCXohP8uqkG5dGi"]['instructions'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Data Organization

For this part, we really have to look at getting all of the recipes into the same format: so the model will learn to recognize the format, and start to use it

In [30]:
pass

# Language Recognition

With some help from: [TowardsDataScience](https://towardsdatascience.com/text-generation-with-python-and-gpt-2-1fecbff1635b)

### Sequence
Here we define an arbitrary recipe string sequence to initially train and test our model with. This text will be improved when we have acquired some data, but this will do for initial development:

In [46]:
sequence = """Ingredients: 3 tomatoes, garlic, spaghetti, Spanish chili's, cheese. Boil the pasta, then rinse. Cut chili's and mince garlic, add to pan
Add tomatoes and the pasta, and grate a generous amount of cheese
Serve with a touch of basil and a glass of good red wine
           """

seqdata = []
print(len(data3.keys()))
count = 0
for key, value in data3.items():
    check = False
    
    #check whether it has the key ingredients
    if "ingredients" in value.keys():
        check = True
    
    #amount of recipes you want
    if count <= 800 and check:
        count = count +1
        listingr = "ingredients: "
        for i in value["ingredients"]:
            listingr = listingr + i + " "
        if type(value["instructions"]) == str:
            r = listingr + value["instructions"]
            seqdata.append(r)
            continue

print(type(seqdata))
#first two recipes
print(seqdata[2])

60039
<class 'list'>
ingredients: 1 1/2 cups dried black beans, picked over and rinsed 8 cups water, plus 1/4 cup 2 cloves garlic 3 tablespoons vegetable oil 1 large green bell pepper, finely chopped, reserving about 1 teaspoon for garnish 1 small red onion, finely chopped, reserving about 1 teaspoon for garnish 2 teaspoons ground cumin 2 tablespoons cider vinegar Salt and pepper 1 plum tomato, finely chopped 1 small red chile, finely chopped Fresh cilantro leaves, for garnish Dollop sour cream, for garnish Serving Suggestion: Tortilla chips In a large saucepan, let the beans soak in enough cold water to cover for 1 hour and drain. Return the beans to the pan with the 8 cups of water and garlic. Bring the water to a boil, lower the heat, and simmer until tender, about 1 hour.
Meanwhile, heat the oil in a large heavy skillet over moderate heat. Add the pepper, onion, and cumin and cook, stirring, until softened.
Drain the beans and reserve 1/2 cup of them. Add the remaining beans to the

In [5]:
#We use the pretrained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

GPT2's development can be found at: Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2018). Language Models are Unsupervised Multitask Learners. 24.

In [50]:
#Tokenizing the inputs
#print(seqdata[1])
inputs = tokenizer.encode(sequence, return_tensors='pt')
#print(inputs)

tokenizer.pad_token = "tokenizer.eos_token"
#padding and truncation to make all tensors same size.
#encoded_choices = [tokenizer.encode(s) for s in seqdata]
input_ids = []

for i in seqdata:
    input_ids.append(tokenizer.encode(i, return_tensors='pt', padding=True,truncation = True))

print(input_ids) #input Ids is now list with tensors of each datapoint

[tensor([[  278, 23320,    25,   352,    14,    17,  6508, 18725,  1924,    11,
         32566, 20720,   352,  1402,  4077, 13385, 32566, 20720,   352,    14,
            17,  6508, 32566, 26790,  4077, 24681,   352,    14,    19,  6508,
         20720, 13544,  1636,   352, 14896, 32202, 41495,   352,   352,    14,
            19, 14180, 36076,   306, 18577,  8469,   263,  1067, 18146,   352,
            14,    17, 22326,  8268,   513,    14,    19, 47234,  5894, 33694,
         16189,  3024, 10746,   352,    14,    19,  6508,  4334,  8566,   352,
            14,    17,  6508, 24178,  9215,   309,   793,  9391, 15376,   290,
         24556,   656,   257,   475,  4400, 16871,  9433,    13,  5849,   351,
          3224, 18577,  8469,   263,  1067, 18146,    11,   290, 14093,   351,
         24178,  9215,    13, 38493,   287,   257,   662,   258,   515,   379,
         13803,  7370, 14361,   329,  1679,   284,  1542,  2431,   393,  1566,
          8675,  1286,  7586,   276,    13]]), tens

# Language Generation

Here we generate a recipe based on the encoded cooking texts that we fed the model above. We will use the model.generate() function to do this.
Model.generate() has a lot of different variable options worth looking at, which can be used to optimize our model. For instance, the temperature variable can be tweaked between 0-5 for increased randomness, we can try no_repeat_ngram_size=2 to prevent repitition, or tweak top_k. We'll just have to play around a bit and see what works best

In [51]:
import pandas as pd
import numpy as np
# We set the output length to 500 tokens
#EncodedRecipe = model.generate(inputs, max_length=500, do_sample=True)
#print(EncodedRecipe)

# We set the output length to 500 tokens

#this only works only when you pick a specific tensor of input ids and not for all
EncodedRecipe2 = model.generate(input_ids[2], max_length=1024, do_sample=True)
print(EncodedRecipe2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[  278, 23320,    25,   352,   352,    14,    17, 14180, 16577,  2042,
         16567,    11,  6497,   625,   290,   374,  1040,   276,   807, 14180,
          1660,    11,  5556,   352,    14,    19,  6508,   362, 45025, 17644,
           513, 30064, 20236,  3056,   352,  1588,  4077,  8966, 13385,    11,
         32566, 20720,    11,  5463,   278,   546,   352, 22326,   329, 36167,
           680,   352,  1402,  2266, 21670,    11, 32566, 20720,    11,  5463,
           278,   546,   352, 22326,   329, 36167,   680,   362, 47234,  2323,
           269,  7230,   362, 30064, 36930, 26600, 13754,   290, 13385,   352,
         22802, 24240,    11, 32566, 20720,   352,  1402,  2266,   442,   576,
            11, 32566, 20720, 20138,   269, 48311,  5667,    11,   329, 36167,
           680, 22209,   404, 11348,  8566,    11,   329, 36167,   680, 49208,
         35042,   295,    25, 28467,  5049, 12014,   554,   257,  1588, 10746,
          6839,    11,  1309,   262, 16567, 33182,  

In [54]:
#Decode the tokenized outputs
#recipe = tokenizer.decode(EncodedRecipe[0], skip_special_tokens=True)
#We filter out the initial context sequence given to the model (258 characters)
#print(recipe[259:])

#Decode the tokenized outputs
recipe2 = tokenizer.decode(EncodedRecipe2[0], skip_special_tokens=True)
#We filter out the initial context sequence given to the model (258 characters)
print(len(recipe2))
print(recipe2) #part of it original and part of it the same as before

1569
ingredients: 1 1/2 cups dried black beans, picked over and rinsed 8 cups water, plus 1/4 cup 2 cloves garlic 3 tablespoons vegetable oil 1 large green bell pepper, finely chopped, reserving about 1 teaspoon for garnish 1 small red onion, finely chopped, reserving about 1 teaspoon for garnish 2 teaspoons ground cumin 2 tablespoons cider vinegar Salt and pepper 1 plum tomato, finely chopped 1 small red chile, finely chopped Fresh cilantro leaves, for garnish Dollop sour cream, for garnish Serving Suggestion: Tortilla chips In a large saucepan, let the beans soak in enough cold water to cover for 1 hour and drain. Return the beans to the pan with the 8 cups of water and garlic. Bring the water to a boil, lower the heat, and simmer until tender, about 1 hour.
Meanwhile, heat the oil in a large heavy skillet over moderate heat. Add the pepper, onion, and cumin and cook, stirring, until softened.
Drain the beans and reserve 1/2 cup of them. Add the remaining beans to the pepper mixture 

this repetition in this recipe is a great example of why we may have to try ways to prevent repetition in the model generation

## Saving Recipes

In [51]:
#Defining a function to save generated recipes as .txt files:
def save(recipe, filename):
    text_file = open(filename, "w", encoding = 'utf8')
    n = text_file.write(recipe)
    text_file.close()

In [52]:
#Saving the previously generated recipe:
save(recipe[259:], 'FirstRecipe.txt')
#Note: I am leaving out the 259 context characters

# Reinforcement learning
I had some inspiration for a potentially novel approach to take this model in: reinforcement learning. While the current model already performs relatively well on creating sensical recipe texts, the model still has absolutely no idea of 'taste', and which ingredients work well together and which don't. Therefore, it may be desireable to give the model some sort of loss function score on its recipes so it knows which recipes are good and which are bad, so the model can improve.
We could do this by either reading the recipes and rating them on how good we think they might be, or maybe even try to cook some of the things our model suggests and see how well they work, so we can help the model improve. While this may take some time and make the coding harder, it could be really cool to have a recipe generation model trained and optimized on actual real-world cooking, and I can hardly imagine anyone has ever done something like that before. Who knows, maybe the model will actually end up becoming a really good cook

Contributors: Emilia, Sergio, Tim